### Building advanced RAG applications using LlamaIndex
LlamaIndex is a powerful datamanagement tool for LLM applications, it allows developers load, transform, index and search over very large datasets which are then added to the LLM to generate more meaningful synthesis from the language model

In [1]:
from llama_index.readers.file.base import SimpleDirectoryReader

In [2]:
documents = SimpleDirectoryReader(input_files=["./MIV2 - LLM paper.pdf"]).load_data()

In [3]:
len(documents)

17

In [4]:
print(documents[0].text)

Human-Robot interaction through joint robot planning
with Large Language Models
Kosi Asuzu1*
1*Birmingham City University.
Abstract
Large Language Models (LLMs) have demonstrated remarkable zero-shot generalisation capa-
bilities, expanding their utility beyond natural language processing into various applications.
Leveraging extensive web knowledge, these models generate meaningful text data in response
to user-defined prompts, introducing a novel mode of interaction with software applications.
Recent investigations have extended the generalizability of LLMs into the domain of robotics,
addressing challenges in existing robot learning techniques such as reinforcement learning and
imitation learning. This paper explores the application of LLMs for robot planning as an alter-
native approach to generate high-level robot plans based on prompts provided to the language
model. The proposed methodology facilitates continuous user interaction and adjustment of task
execution plans in real-ti

In [5]:
print(type(documents), "\n")
print(len(documents), "\n")
print(type(documents[0]))
print(documents[0])

<class 'list'> 

17 

<class 'llama_index.schema.Document'>
Doc ID: 26029736-135b-4824-bc41-107558227130
Text: Human-Robot interaction through joint robot planning with Large
Language Models Kosi Asuzu1* 1*Birmingham City University. Abstract
Large Language Models (LLMs) have demonstrated remarkable zero-shot
generalisation capa- bilities, expanding their utility beyond natural
language processing into various applications. Leveraging extensive
web knowl...


### Building a simple RAG pipeline with llama index

In [6]:
from llama_index.schema import Document
document  = Document(text="\n\n".join([ doc.text for doc in documents]))

In [7]:
print(len(document.text))

54037


In [8]:
print(document.text[-1000:])

lucci, I. and Arezes, P.M., 2019. A brief overview of the use of
collaborative robots in industry 4.0: Human role and safety. Occupational and environmental
safety and health, pp.641-650.
[55] Bragan¸ ca, S., Costa, E., Castellucci, I. and Arezes, P.M., 2019. A brief overview of the use of
collaborative robots in industry 4.0: Human role and safety. Occupational and environmental
safety and health, pp.641-650.
[56] Park, J.S., O’Brien, J., Cai, C.J., Morris, M.R., Liang, P. and Bernstein, M.S., 2023, October.
Generative agents: Interactive simulacra of human behavior. In Proceedings of the 36th Annual
ACM Symposium on User Interface Software and Technology (pp. 1-22).
Appendix A Section title of first appendix
An appendix contains supplementary information that is not an essential part of the text itself but
which may be helpful in providing a more comprehensive understanding of the research problem or
it is information that is too cumbersome to be included in the body of the paper.
17

In [9]:
from llama_index.indices.vector_store import VectorStoreIndex
from llama_index.service_context import ServiceContext
from llama_index.llms.openai import OpenAI

In [10]:
from dotenv import load_dotenv
load_dotenv()

True

In [11]:
mistral7b = OpenAI(temperature=0.0, model="mistralai/Mistral-7B-Instruct-v0.2")

In [12]:
mistral7b.to_dict()

{'system_prompt': None,
 'pydantic_program_mode': <PydanticProgramMode.DEFAULT: 'default'>,
 'model': 'mistralai/Mistral-7B-Instruct-v0.2',
 'temperature': 0.0,
 'max_tokens': None,
 'additional_kwargs': {},
 'max_retries': 3,
 'timeout': 60.0,
 'default_headers': None,
 'reuse_client': True,
 'api_base': 'https://api.together.xyz/v1',
 'api_version': '',
 'class_name': 'openai_llm'}

In [13]:
service_context = ServiceContext.from_defaults(
    llm=mistral7b, embed_model="local:BAAI/bge-small-en-v1.5"
)

/Users/kosisochukwuasuzu/Developer/ai-startups/test-demos/pdfchat/venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [14]:
index = VectorStoreIndex.from_documents([document],
                                        service_context=service_context)

In [15]:
query_engine = index.as_query_engine()

In [20]:
response = query_engine.query("What is the title of the paper")

In [21]:
print(str(response))

The title of a paper cannot be determined from the context information provided as each paper is identified by its unique title.


### Evaluation using TruLens

In [ ]:
eval_questions = []
with open('eval_questions.txt', 'r') as file:
    for line in file:
        # Remove newline character and convert to integer
        item = line.strip()
        print(item)
        eval_questions.append(item)

In [ ]:
new_question = "What is the right AI job for me?"
eval_questions.append(new_question)

In [ ]:
print(eval_questions)

In [ ]:
from trulens_eval import Tru
tru = Tru()

tru.reset_database()

### Building Advanced RAG applications
The powerful part about llama index is its vast retrieval techinques implemented for advanced retrieval usage

#### Sentence Window Retreival
Sentence window retrieval is a RAG technique that handles fetching surrounding context when querying the embedding space, this helps gather surroundign information when queried

In [ ]:
from utils import build_sentence_window_index

In [ ]:
sentence_index = build_sentence_window_index(
    document,
    mistral7b,
    embed_model="local:BAAI/bge-small-en-v1.5",
    save_dir="sentence_index"
)

In [ ]:
from utils import get_sentence_window_query_engine

sentence_window_engine = get_sentence_window_query_engine(sentence_index)

In [ ]:
window_response = sentence_window_engine.query(
    "how do I get started on a personal project in AI?"
)
print(str(window_response))

In [ ]:
tru.reset_database()

tru_recorder_sentence_window = get_prebuilt_trulens_recorder(
    sentence_window_engine,
    app_id = "Sentence Window Query Engine"
)

In [ ]:
for question in eval_questions:
    with tru_recorder_sentence_window as recording:
        response = sentence_window_engine.query(question)
        print(question)
        print(str(response))

In [ ]:
tru.get_leaderboard(app_ids=[])

In [ ]:
tru.run_dashboard()

### Automerging retrieval
Merges similar retrieval contents for more accurate retreival process

In [ ]:
from utils import build_automerging_index

automerging_index = build_automerging_index(
    documents,
    mistral7b,
    embed_model="local:BAAI/bge-small-en-v1.5",
    save_dir="merging_index"
)

In [ ]:
from utils import get_automerging_query_engine

automerging_query_engine = get_automerging_query_engine(
    automerging_index,
)

In [ ]:
auto_merging_response = automerging_query_engine.query(
    "How do I build a portfolio of AI projects?"
)
print(str(auto_merging_response))

In [ ]:
tru.reset_database()

tru_recorder_automerging = get_prebuilt_trulens_recorder(automerging_query_engine,
                                                         app_id="Automerging Query Engine")

In [ ]:
for question in eval_questions:
    with tru_recorder_automerging as recording:
        response = automerging_query_engine.query(question)
        print(question)
        print(response)

In [ ]:
tru.get_leaderboard(app_ids=[])

In [ ]:
# launches on http://localhost:8501/
tru.run_dashboard()